In [685]:
import torch 
from torch import nn

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [686]:
df = pd.read_csv('train_data/train.csv')

In [687]:
#o objetivo aqui vai ser transformar o meu dataframe em tensor, sendo a primeira coluna o label que eu quero descobrir e o resto o data
df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [688]:
#pratica do markdown acima
#repare que label é um vetor e data é uma matriz
label = df['label'].values
data = df.drop('label', axis=1).values


In [689]:
scaler = StandardScaler()
data = scaler.fit_transform(data)

In [690]:
#transforming data and label into tensors
label = torch.from_numpy(label).type(torch.float32)
data = torch.from_numpy(data).type(torch.float32).view(-1, 1, 28, 28)


In [691]:
print(label.shape, data.shape)

torch.Size([42000]) torch.Size([42000, 1, 28, 28])


In [692]:
#creating dataloader and dataset 

dataset = TensorDataset(data, label)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [693]:
#it has 10 classes and the input shape of 28*28
image, label = dataset[0]
image.shape

torch.Size([1, 28, 28])

In [694]:
import torch
import torch.nn as nn

class DigitModel(nn.Module):
    
    def __init__(self, input_feat: int, hidden_units: int, output_feat: int):
        super(DigitModel, self).__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_feat, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25) #dropout has to be used with subsequencial layers
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25) #dropout has to be used with subsequencial layers
        )
        self.conv_block_3 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25)
        )
        self.conv_block_4 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25)
        )
    
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*3*3, out_features=hidden_units),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features=hidden_units, out_features=output_feat) #dropout has to be used with subsequencial layers
            
            
        )

    def forward(self, x):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.conv_block_3(x)
        x = self.classifier(x)
        return x



In [695]:
device = 'cuda'

In [696]:
#1 cause theres only black and white, output feat of 10 cause there's 10 digits
model_0 = DigitModel(input_feat=1, hidden_units=128, output_feat=10).to(device)

In [697]:
#loss and optmizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

In [698]:
#building a function to train our model
from tqdm import tqdm

def training_model(epochs, model, dataloader):

    for epoch in tqdm(range(epochs)):

        #train mode 
        model.train()

        #track the training loss
        train_loss = 0
        correct = 0
        total = 0

        for batch, (inputs, targets) in enumerate(dataloader):

            #move the tensors to cuda for better performance
            
            inputs = inputs.to(device) #you have to turn into this dimension, because conv2d only accepts 3d and 4d
            targets = targets.to(device).long()
            
            #zero grad optmizer
            optimizer.zero_grad()

            #forward pass
            y_pred = model(inputs)

            #calculate the loss
            loss = loss_fn(y_pred, targets)
            train_loss +=loss.item()

            #backprop
            loss.backward()

            #apply the optmizer
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            _, predicted = torch.max(y_pred.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()




            if batch % 200 == 0:
                print(f'Batch: {batch}, Loss: {loss.item()}')
    #loss for each epoch

        train_accuracy = 100 * correct / total
        avg_train_loss = train_loss/len(dataloader)
        print(f'Epoch: {epoch}, Train Loss: {avg_train_loss}, Accuracy: {train_accuracy}')
        
    return model, avg_train_loss, train_accuracy
        




In [708]:
model_0_5, train_loss_model0_5, train_acc_model_0_5 = training_model(epochs=7, model=model_0, dataloader=dataloader)

  0%|          | 0/7 [00:00<?, ?it/s]

Batch: 0, Loss: 0.03128712996840477
Batch: 200, Loss: 6.045338523108512e-05
Batch: 400, Loss: 0.01913374848663807
Batch: 600, Loss: 0.01060737855732441
Batch: 800, Loss: 0.0027464560698717833
Batch: 1000, Loss: 0.34076988697052
Batch: 1200, Loss: 0.00046646472765132785


 14%|█▍        | 1/7 [00:19<01:55, 19.26s/it]

Epoch: 0, Train Loss: 0.05899594381047736, Accuracy: 98.73333333333333
Batch: 0, Loss: 0.22220459580421448
Batch: 200, Loss: 2.2966021788306534e-05
Batch: 400, Loss: 0.20192398130893707
Batch: 600, Loss: 0.0006886441842652857
Batch: 800, Loss: 0.0377645306289196
Batch: 1000, Loss: 0.01058010570704937
Batch: 1200, Loss: 0.008200050331652164


 29%|██▊       | 2/7 [00:38<01:35, 19.12s/it]

...

Epoch: 4, Train Loss: 0.055549944984417274, Accuracy: 98.98809523809524
Batch: 0, Loss: 4.35458332503913e-06
Batch: 200, Loss: 0.0007412965060211718
Batch: 400, Loss: 1.0207186278421432e-06
Batch: 600, Loss: 9.350395657747868e-07
Batch: 800, Loss: 0.18584704399108887
Batch: 1000, Loss: 0.007899417541921139
Batch: 1200, Loss: 0.013487712480127811


 86%|████████▌ | 6/7 [01:56<00:19, 19.48s/it]

Epoch: 5, Train Loss: 0.04985067353840345, Accuracy: 99.0547619047619
Batch: 0, Loss: 0.0014601665316149592
Batch: 200, Loss: 0.27243682742118835
Batch: 400, Loss: 0.05785893648862839
Batch: 600, Loss: 0.09314566850662231
Batch: 800, Loss: 0.00023093588242772967
Batch: 1000, Loss: 0.002583958674222231
Batch: 1200, Loss: 0.12702156603336334


100%|██████████| 7/7 [02:16<00:00, 19.51s/it]

Epoch: 6, Train Loss: 0.04930038909397533, Accuracy: 99.04761904761905


In [702]:
# Carregar os dados de teste
test_df = pd.read_csv('train_data/test.csv')
test_data = test_df.values

# Normalizar os dados de teste
test_data = scaler.transform(test_data)

# Converter para tensores PyTorch
test_data = torch.from_numpy(test_data).type(torch.float32).view(-1, 1, 28, 28)

# Criar DataLoader para o conjunto de dados de teste
test_dataset = TensorDataset(test_data)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [709]:
# Função para fazer previsões
def predict(model, dataloader, device):
    model.eval()  # Colocar o modelo em modo de avaliação
    predictions = []
    
    with torch.no_grad():
        for batch in dataloader:
            inputs = batch[0].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
    
    return predictions

In [710]:
test_predictions = predict(model_0_5, test_loader, device)


In [711]:
test_predictions

[2,
 0,
 9,
 9,
 3,
 7,
 0,
 3,
 0,
 3,
 5,
 7,
 4,
 0,
 4,
 3,
 3,
 1,
 9,
 0,
 9,
 1,
 1,
 5,
 7,
 4,
 2,
 7,
 4,
 7,
 7,
 5,
 4,
 2,
 6,
 2,
 5,
 5,
 1,
 6,
 7,
 7,
 4,
 9,
 8,
 7,
 8,
 2,
 6,
 7,
 6,
 8,
 8,
 3,
 8,
 2,
 1,
 2,
 2,
 0,
 4,
 1,
 7,
 0,
 0,
 0,
 1,
 9,
 0,
 1,
 6,
 5,
 8,
 8,
 2,
 8,
 9,
 9,
 2,
 3,
 5,
 4,
 1,
 8,
 9,
 2,
 4,
 3,
 6,
 7,
 2,
 0,
 6,
 6,
 1,
 4,
 3,
 9,
 7,
 4,
 0,
 9,
 2,
 0,
 7,
 3,
 0,
 5,
 0,
 8,
 0,
 0,
 4,
 7,
 1,
 7,
 1,
 1,
 3,
 3,
 3,
 7,
 2,
 8,
 6,
 3,
 8,
 7,
 7,
 4,
 3,
 5,
 6,
 0,
 0,
 0,
 3,
 1,
 3,
 6,
 4,
 3,
 4,
 5,
 5,
 8,
 7,
 7,
 2,
 8,
 4,
 3,
 5,
 6,
 5,
 3,
 7,
 5,
 7,
 8,
 3,
 0,
 4,
 5,
 1,
 3,
 7,
 6,
 3,
 0,
 2,
 7,
 8,
 6,
 1,
 3,
 7,
 4,
 1,
 2,
 4,
 8,
 5,
 2,
 4,
 9,
 2,
 1,
 6,
 0,
 6,
 1,
 4,
 9,
 6,
 0,
 9,
 7,
 6,
 9,
 1,
 9,
 0,
 9,
 9,
 0,
 8,
 4,
 6,
 2,
 0,
 9,
 3,
 6,
 3,
 2,
 1,
 6,
 3,
 4,
 2,
 3,
 1,
 2,
 2,
 0,
 4,
 6,
 1,
 0,
 0,
 4,
 9,
 1,
 7,
 3,
 2,
 3,
 8,
 6,
 8,
 6,
 2,
 8,
 5,
 5,
 4,
 8,
 3,
 5,


In [712]:
submission_df = pd.DataFrame({
    'ImageId': list(range(1, len(test_predictions) + 1)),
    'Label': test_predictions
})

In [713]:
# Salvar o DataFrame como um arquivo CSV
submission_df.to_csv('submission.csv', index=False)

print("Previsões salvas no arquivo 'submission.csv'.")

Previsões salvas no arquivo 'submission.csv'.
